# Model Context Protocol (MCP)
MCP는 유연한 LLM 어플리케이션 통합을 위한 프로토콜

## Client-Server Architecture
MCP는 Client-Server 구조를 따르고, 3가지 컴포넌트로 구성됨.
- Host : LLM 어플리케이션
- Clinet : Host속에 존재해서 Server와 1:1 연결을 맺는 구성요소
- Server : Context, Tools, Prompt를 Client에 제공


## Protocol layer
프로토콜 레이어는 통신 패턴을 다루는데 사용됨

## Transport layer
전송 계층은 실질적인 커뮤니케이션이 일어나는 파트로, JSON형식의 데이터를 교환함.
두가지 통신 매커니즘이 있음.
- Stdio : 로컬 환경에서 사용
- HTTP with SSE : 클라이언트가 서버에 POST를 하는 경우, 서버측에서 클라이언트에 메세지를 보내는 경우

## Connection Lifecycle 
1. Initialzation : 3-way 핸드쉐이크처럼 초기 통신 단계
2. Message Exchange : 메세지를 교환하는 부분으로 2가지 패턴이 주요함 
- Request-Response : 클라이언트-서버간 request/response 통신
- Notifications : 1방향 메세지
3. Termination : 연결 종료

In [1]:
import asyncio
import mcp.types as types
from mcp.server import Server
from mcp.server.stdio import stdio_server

app = Server("example-server")

@app.list_resources()
async def list_resources() -> list[types.Resource]:
    return [
        types.Resource(
            uri="example://resource",
            name="Example Resource"
        )
    ]

async def main():
    async with stdio_server() as streams:
        await app.run(
            streams[0],
            streams[1],
            app.create_initialization_options()
        )

if __name__ == "__main__":
    asyncio.run(main())

ModuleNotFoundError: No module named 'mcp'

## Resources
LLM에 사용될 데이터를 가져오는 개념. 
클라이언트가 리소스를 어떻게, 언제 사용할지 결정.

### 데이터의 종류
- 파일
- 데이터베이스
- API 
- 이미지 등

### Resource URI
리소스 파일은 다음 형식으로 구성
```
# format
[protocol]://[host]/[path]
# example
file:///home/user/documents/report.pdf
screen://localhost/display1
```

### Resource Types
- Text resources (UTF-8): source code , json file, plain text, and etc
- Binary resources (base64) : Image, PDF, Vedio, and etc   

### Resource Method
We can use two main methods to use data
- resources/list : direct access to resources
```
{
  uri: string;           // Unique identifier for the resource
  name: string;          // Human-readable name
  description?: string;  // Optional description
  mimeType?: string;     // Optional MIME type
}
```

- resources/read : The server responds with a list of resource contents.
```
{
  contents: [
    {
      uri: string;        // The URI of the resource
      mimeType?: string;  // Optional MIME type

      // One of:
      text?: string;      // For text resources
      blob?: string;      // For binary resources (base64 encoded)
    }
  ]
}
```

### Updating Resource
Two methos supprots real-time data update :
- notifications/resources/list_changed : Server can notify clients when list of resources are changed
- resources/subscribe : client can subsribe server resources by using with resource URI
- resources/read : client can fetch lastest resource
- resources/unsubscribe : client can unsubscribe

```
app = Server("example-server")

@app.list_resources()
async def list_resources() -> list[types.Resource]:
    return [
        types.Resource(
            uri="file:///logs/app.log",
            name="Application Logs",
            mimeType="text/plain"
        )
    ]

@app.read_resource()
async def read_resource(uri: AnyUrl) -> str:
    if str(uri) == "file:///logs/app.log":
        log_contents = await read_log_file()
        return log_contents

    raise ValueError("Resource not found")

# Start server
async with stdio_server() as streams:
    await app.run(
        streams[0],
        streams[1],
        app.create_initialization_options()
    )
```


# Prompt
재활용 가능한 프롬프트와 워크플로우를 만드는 컨셉
MCP에서의 프롬프트가 할 수 있는 행위:
- 다양한 인자를 사용
- 컨텍스트에 리소스 포함
- 여러 행위를 묶음
- 워크플로우에 대한 가이드
- UI

Endpoints
- prompts/list : 사용가능한 프롬프트 목록
- prompts/get : 사용할 프롬프트를 가져옴
- 